In [2]:
print("hello");

hello


In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# List of URLs to load documents from
urls = [
    "<https://lilianweng.github.io/posts/2023-06-23-agent/>",
    "<https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/>",
    "<https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/>",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

USER_AGENT environment variable not set, consider setting it to identify your requests.


ModuleNotFoundError: No module named 'bs4'

In [10]:
from langchain_community.document_loaders import PyPDFLoader

# Specify the path to your PDF file
pdf_path = "./test.pdf"

# Create a PDF loader
loader = PyPDFLoader(pdf_path)

# Load the document into LangChain
documents = loader.load()

# Check the loaded content
for doc in documents:
    print(doc.page_content)

BANGLADESH RAILWAY
বাংলােদশ রলওেয়
Dear TAHSIN AHMED MAJUMDER,
Your request to book e-ticket for your journey in Bangladesh Railway was successful. You can travel on the train mentioned in
the ticket subject to showing your NID or Photo ID card. The details of your e-ticket are as below:
বাংলােদশ রলওেয়েত মেণর জ আপনার চািহত ই-িটিকট সফলভােব দান করা হেয়েছ। আপনার এনআইিড িকংবা ছিব সিলত আইিড দখােনা সােপে
আপিন িটিকেট বিণত েন যাা করেত পারেবন। ই-িটিকেটর িবািরত িনে দয়া হল:-
Journey Information (যাার তথ)
Issue Date & Time (দােনর তািরখ ও সময়) 25-10-2024 11:59 (২৫-১০-২০২৪ ১১:৫৯)
Journey Date & Time (যাার তািরখ ও সময়) 26-10-2024 18:38 (২৬-১০-২০২৪ ১৮:৩৮)
Train Name & Number (ন নর ও নাম) CHATTALA EXPRESS [802] (চলা এেস [৮০২])
From Station (ারিক শন) Feni (ফনী)
To Station (গব শন) Chattogram (চাম)
Class Name (িণর নাম) SNIGDHA (িা)
Coach Name / Seat(s) (কােচর নাম / আসন) UMA-47, UMA-46  (ঙ-৪৭, ঙ-৪৬)
No. of Seats (আসন সংখা) 2 (২)
No. of Adult Passenger(s) (াবয় যা

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(documents)

In [13]:
print(doc_splits)

[Document(metadata={'source': './test.pdf', 'page': 0}, page_content='BANGLADESH RAILWAY\nবাংলােদশ \ue04eরলওেয়\nDear TAHSIN AHMED MAJUMDER,\nYour request to book e-ticket for your journey in Bangladesh Railway was successful. You can travel on the train mentioned in\nthe ticket subject to showing your NID or Photo ID card. The details of your e-ticket are as below:'), Document(metadata={'source': './test.pdf', 'page': 0}, page_content='বাংলােদশ \ue04eরলওেয়েত \ue0bfমেণর জ\ue0ae আপনার চািহত ই-িটিকট সফলভােব \ue0b3দান করা হেয়েছ। আপনার এনআইিড িকংবা ছিব স\ue0c3িলত আইিড \ue04eদখােনা সােপে\ue058'), Document(metadata={'source': './test.pdf', 'page': 0}, page_content='আপিন িটিকেট বিণ\ue04dত \ue04e\ue07fেন যা\ue093া করেত পারেবন। ই-িটিকেটর িব\ue0e8ািরত িনে\ue0c0 \ue04eদয়া হল:-\nJourney Information (যা\ue093ার তথ\ue04a)'), Document(metadata={'source': './test.pdf', 'page': 0}, page_content='Issue Date & Time (\ue0b3দােনর তািরখ ও সময়) 25-10-2024 11:59 (২৫-১০-২০২৪ ১১:৫৯)\nJourney Date & Time (যা\ue09

In [18]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [21]:
embeddings = OllamaEmbeddings(model='mxbai-embed-large')

# Convert documents to vectors and index vectors
db = FAISS.from_documents(doc_splits, embeddings)

In [22]:
retriever = db.as_retriever(k=4)

In [23]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [24]:
# Initialize the LLM with Llama 3.1 model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [25]:
rag_chain = prompt | llm | StrOutputParser()

In [26]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [ ]:
rag_application = RAGApplication(retriever, rag_chain)
question = "What is the fare"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: What is the fare
Answer: The fare is BDT 360.00 (৩৬০.০০ টাকা). Additionally, there are VAT and Service Charge of BDT 54.00 and BDT 40.00 respectively. The total fare comes out to be BDT 454.00 (৪৫৪.০০ টাকা).
